### 1. Wahl zweier großer Primzahlen

In [19]:
import random
from math import gcd

In [20]:
def miller_rabin(n, k=8):
    """
    Sources: 
        https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test
        https://github.com/kaushiksk/rsa-from-scratch
    """
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False
    
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    def iscomposite(a):
        x = pow(a, d, n)
        if x == 1 or x == n-1:
            return False
        for _ in range(s):
            x = (x**2) % n
            if x == n-1:
                return False
        return True
    
    for _ in range(k):
        a = random.randint(2, n-1)
        if iscomposite(a):
            return False
    return True
"""
for i in range(1, 100):
    if miller_rabin(i):
        print(i, end=" ")
"""


'\nfor i in range(1, 100):\n    if miller_rabin(i):\n        print(i, end=" ")\n'

In [21]:
def generateprime(bits=100):
    p = random.getrandbits(bits)
    if not p % 2 == 1:
        p += 1
    while not miller_rabin(p):
        p += 2
    return p

#generateprime(100)

In [22]:
q = generateprime()
p = generateprime()
while q == p:
    p = generateprime()

### 2. Berechnung des Moduls N

In [23]:
n = q * p

### 3. Berechnung der Eulerschen Phi-Funktion von N

In [24]:
phi = (p - 1) * (q - 1)

### 4. Wahl eines Exponenten e

In [25]:
e = 65537
while gcd(e, phi) != 1:
    e = random.randint(2, phi-1)

### 5. Berechnung des privaten Schlüssels d

In [26]:
def xgcd(e, phi):
    if e == 0:
        return phi, 0, 1
    
    g, x1, y1 = xgcd(phi%e, e)

    x = y1 - (phi//e) * x1
    y = x1

    return g, x, y

def inverse(e, phi):
    g, x, y = xgcd(e, phi)
    return (x%phi + phi) % phi

d = inverse(e, phi)

public_key = (e, n)
private_key = (d, n)

print(f"Public Key: {public_key}, Private Key: {private_key}")


Public Key: (65537, 296409672140808916660644165697174455370655867775579221203391), Private Key: (85145925015226035141268093800356865600596175877699542935873, 296409672140808916660644165697174455370655867775579221203391)


## Verschlüsselung und Entschlüsselung

In [ ]:
m = "hallo"
ascii_value = [ord(i) for i in m]
#print(ascii_value)

[104, 97, 108, 108, 111]


### Verschlüsselung

In [28]:
def encode(m, public_key):
    e = public_key[0]
    n = public_key[1]
    return pow(m, e, n)

c = [encode(m, public_key) for m in ascii_value]
c

[80847849026894272364299089673213378188430636424388960936076,
 128133224288325922734782082170287314307435049121702390570954,
 152029485094569774139142917578974501285738424717376986087719,
 152029485094569774139142917578974501285738424717376986087719,
 251729048083260835500682302414245878382851168191240086774547]

### Entschlüsselung

In [29]:
def decode(c, private_key):
    d = private_key[0]
    n = private_key[1]
    return pow(c, d, n)

m = [decode(i, private_key) for i in c]
print(m)
m = "".join(chr(i) for i in m)
print(m)

[104, 97, 108, 108, 111]
hallo
